In [132]:
'''
Add the code below the TO-DO statements to finish the assignment. Keep the interfaces
of the provided functions unchanged. Change the returned values of these functions
so that they are consistent with the assignment instructions. Include additional import
statements and functions if necessary.
'''

import csv
import numpy as np
import random
'''
The loss functions shall return a scalar, which is the *average* loss of all the examples
'''

'''
For instance, the square loss of all the training examples is computed as below:

def squared_loss(train_y, pred_y):

    loss = np.mean(np.square(train_y - pred_y))

    return loss
'''

def logistic_loss(train_y, pred_y):

    # TO-DO: Add your code here

    return np.mean(np.log(1+np.exp(-train_y*pred_y)))

def hinge_loss(train_y, pred_y):
   
    # TO-DO: Add your code here
    hinge = np.mean(np.maximum(0,(1-train_y*pred_y)))
    return hinge

'''
The regularizers shall compute the loss without considering the bias term in the weights
'''
def l1_reg(w):

    # TO-DO: Add your code here

    return np.linalg.norm(w[:-1], ord=1)

def l2_reg(w):

    # TO-DO: Add your code here

    return np.linalg.norm(w[:-1], ord=2)

def train_classifier(train_x, train_y, learn_rate, loss, lambda_val=None, regularizer=None):

    # TO-DO: Add your code here
    """create a weights vector of random normal values"""
    weights =  np.random.normal(scale=0.01, size = len(train_x[0]))
    numIters = 150
    """Do Gradient descent numIters time"""
    for i in range(numIters):
        gradient = calculateGradient(weights, train_x, train_y, learn_rate, loss, lambda_val, regularizer)
        #update the weights with each iteration using the gradient
        weights = weights - learn_rate*gradient
    return weights

def calculateGradient(weights, train_x, train_y, learn_rate, loss, lambda_val, regularizer):
    """this function calculates the gradient and returns a gradient vector"""    
    
    """create a score vector that stores the dot product of weights and examples"""
    """calculate the dot product for intitial prediction"""
    score = np.zeros(len(train_x))
   
    for i,x in enumerate(train_x,0):
        score[i]=np.dot(weights, x)
    firstTerm_base = 0
    #Regularizer is None if it is Logistic Regression
    if(regularizer!= None or lambda_val != None):
        firstTerm_base = lambda_val*regularizer(weights)
    secondTerm_base = loss(train_y, score)#/len(train_x)
    base_value = firstTerm_base +secondTerm_base
   
    """Add h to each column of train_x and compute gradient again"""
    h = 0.0001
    gradient = np.zeros(len(weights))
    #print("WEIGHTS", weights)
    for i in range(len(weights)):
        weights[i] = weights[i]+h
        #print("WEIGHTS: ",weights)
        for j,x in enumerate(train_x,0):
            score[j]=np.dot(weights, x)
        firstTerm_new = 0
        if(regularizer != None or lambda_val != None):
            firstTerm_new = lambda_val*regularizer(weights)
        secondTerm_new = loss(train_y, score)
        new_value = firstTerm_new +secondTerm_new
        
        gradient[i] = (new_value - base_value)/h
   
    return gradient
   
def test_classifier(w, test_x):

    # TO-DO: Add your code here
    pred_y = np.dot(test_x, w)
    return pred_y

def compute_accuracy( pred_y, test_y):
    correct = 0
    for i in range(len(pred_y)):
        if ((test_y[i] >0 and pred_y[i] > 0) or (test_y[i] < 0 and pred_y[i] < 0)):
            correct += 1
    return (correct/float(len(test_y))) * 100.0


In [133]:
def main():

    # Read the training data file
    szDatasetPath = 'winequality-white.csv'
    listClasses = []
    listAttrs = []
    bFirstRow = True
    with open(szDatasetPath) as csvFile:
        csvReader = csv.reader(csvFile, delimiter=',')
        for row in csvReader:
            if bFirstRow:
                bFirstRow = False
                continue
            if int(row[-1]) < 6:
                listClasses.append(-1)
                listAttrs.append(list(map(float, row[1:len(row) - 1])))
            elif int(row[-1]) > 6:
                listClasses.append(+1)
                listAttrs.append(list(map(float, row[1:len(row) - 1])))

    dataX = np.array(listAttrs)
    dataY = np.array(listClasses)

# 5-fold cross-validation
# Note: in this assignment, preprocessing the feature values will make
# a big difference on the accuracy. Perform feature normalization after
# spliting the data to training and validation set. The statistics for
# normalization would be computed on the training set and applied on
# training and validation set afterwards.
# TO-DO: Add your code here

   
    #get a random permutation of elements in dataX
    randX = []
    randY = []
    temp = np.arange(0, 2700, 1).tolist()
    random.shuffle(temp)
   
    for i in temp:
        randX.append(dataX[i])
        randY.append(dataY[i])
    randDataX = np.array(randX)
    randDataY = np.array(randY)
   
    #perform k-folds on it
    k = 5
    foldsX = (np.array_split(randDataX, k))
    foldsY = (np.array_split(randDataY, k))
    """Setting the hyper parameter for train_classifier"""
    learningRate = [0.1,0.01,0.001,0.0001,0.00001]
    lambdaValue = [100,10,1,0.1,0.01,0.001]
    
    for lr in learningRate:
        
        for lmbda in lambdaValue:
            
            """Run a loop to get the folds. Fold[i] is the test data and remaining parts are the training data"""
            for i in range(k):
                print("\n\bFOLD #:", i)
                trainX = foldsX.copy() 
                trainY = foldsY.copy()
                testX = foldsX[i]
                testY = foldsY[i]
                """Deleting the test data from the aggregate to perform k-fold cross validation"""
                del trainX[i]
                del trainY[i]

                trainX = np.concatenate(trainX,axis=0)
                trainY = np.concatenate(trainY,axis=0)

                """Normalizing the training data by calculating the mean and std deviation of each feature value and 
                subtracting each value by the mean and dividing by std deviation"""
                mean = []
                std = []
                nTrainX = []
                #print("Orig Dat: ", trainX)
                for j in range(len(trainX[0])):
                    columnList = trainX[:,j]
                    mean.append(np.mean(columnList))
                    std.append(np.std(columnList))
                    columnList= columnList-mean[j]
                    columnList = columnList/std[j]
                    nTrainX.append(columnList)
                nTrainX = np.array(nTrainX).T

                """Normalize the testing data"""
                nTestX = []
                for j in range(len(trainX[0])):
                    columnList = testX[:,j]
                    columnList= columnList-mean[j]
                    columnList = columnList/std[j]
                    nTestX.append(columnList)
                nTestX = np.array(nTestX).T
                print("\nLogistic Regression: ")
                weights = train_classifier(nTrainX, trainY, lr, logistic_loss)
                nPredY= test_classifier(weights,nTestX)
                print("\tLearning Rate: ", lr, "\n\tLambda Value: ", lmbda)
                print("\tAccuracy: ", compute_accuracy(nPredY, testY))
                
                print("\nSVM: ")
                weights = train_classifier(nTrainX, trainY, lr, hinge_loss, lmbda, l2_reg)
                nPredY= test_classifier(weights,nTestX)
                print("\tLearning Rate: ", lr, "\n\tLambda Value: ", lmbda)
                print("\tAccuracy: ", compute_accuracy(nPredY, testY))
    return None

if __name__ == "__main__":

    main()


FOLD #: 0

Logistic Regression: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  79.25925925925927

SVM: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  72.4074074074074

FOLD #: 1

Logistic Regression: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  80.55555555555556

SVM: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  76.48148148148148

FOLD #: 2

Logistic Regression: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  80.74074074074075

SVM: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  38.148148148148145

FOLD #: 3

Logistic Regression: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  79.62962962962963

SVM: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  70.0

FOLD #: 4

Logistic Regression: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  79.44444444444444

SVM: 
	Learning Rate:  0.1 
	Lambda Value:  100
	Accuracy:  70.0

FOLD #: 0

Logistic Regression: 
	Learning Rate:  0.1 
	Lambda Value:  10
	Accuracy:  79.25925

	Learning Rate:  0.01 
	Lambda Value:  1
	Accuracy:  78.88888888888889

FOLD #: 0

Logistic Regression: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  67.77777777777779

SVM: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  77.22222222222223

FOLD #: 1

Logistic Regression: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  70.74074074074073

SVM: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  79.07407407407408

FOLD #: 2

Logistic Regression: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  69.25925925925925

SVM: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  79.07407407407408

FOLD #: 3

Logistic Regression: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  69.07407407407408

SVM: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  75.92592592592592

FOLD #: 4

Logistic Regression: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  71.29629629629629

SVM: 
	Learning Rate:  0.01 
	Lambda Value:  0.1
	Accuracy:  78.3333

	Learning Rate:  0.001 
	Lambda Value:  0.001
	Accuracy:  64.44444444444444

SVM: 
	Learning Rate:  0.001 
	Lambda Value:  0.001
	Accuracy:  59.81481481481481

FOLD #: 4

Logistic Regression: 
	Learning Rate:  0.001 
	Lambda Value:  0.001
	Accuracy:  61.48148148148148

SVM: 
	Learning Rate:  0.001 
	Lambda Value:  0.001
	Accuracy:  59.62962962962963

FOLD #: 0

Logistic Regression: 
	Learning Rate:  0.0001 
	Lambda Value:  100
	Accuracy:  41.66666666666667

SVM: 
	Learning Rate:  0.0001 
	Lambda Value:  100
	Accuracy:  61.29629629629629

FOLD #: 1

Logistic Regression: 
	Learning Rate:  0.0001 
	Lambda Value:  100
	Accuracy:  37.592592592592595

SVM: 
	Learning Rate:  0.0001 
	Lambda Value:  100
	Accuracy:  21.296296296296298

FOLD #: 2

Logistic Regression: 
	Learning Rate:  0.0001 
	Lambda Value:  100
	Accuracy:  38.88888888888889

SVM: 
	Learning Rate:  0.0001 
	Lambda Value:  100
	Accuracy:  58.7037037037037

FOLD #: 3

Logistic Regression: 
	Learning Rate:  0.0001 
	Lambda Va

	Learning Rate:  1e-05 
	Lambda Value:  1
	Accuracy:  55.37037037037037

SVM: 
	Learning Rate:  1e-05 
	Lambda Value:  1
	Accuracy:  53.70370370370371

FOLD #: 2

Logistic Regression: 
	Learning Rate:  1e-05 
	Lambda Value:  1
	Accuracy:  50.37037037037037

SVM: 
	Learning Rate:  1e-05 
	Lambda Value:  1
	Accuracy:  37.03703703703704

FOLD #: 3

Logistic Regression: 
	Learning Rate:  1e-05 
	Lambda Value:  1
	Accuracy:  52.96296296296297

SVM: 
	Learning Rate:  1e-05 
	Lambda Value:  1
	Accuracy:  38.88888888888889

FOLD #: 4

Logistic Regression: 
	Learning Rate:  1e-05 
	Lambda Value:  1
	Accuracy:  47.77777777777778

SVM: 
	Learning Rate:  1e-05 
	Lambda Value:  1
	Accuracy:  42.592592592592595

FOLD #: 0

Logistic Regression: 
	Learning Rate:  1e-05 
	Lambda Value:  0.1
	Accuracy:  46.2962962962963

SVM: 
	Learning Rate:  1e-05 
	Lambda Value:  0.1
	Accuracy:  39.25925925925926

FOLD #: 1

Logistic Regression: 
	Learning Rate:  1e-05 
	Lambda Value:  0.1
	Accuracy:  43.7037037

# This is testing code:

In [72]:
    k = 2
    randDataX = np.array([[1,2,3,4,5,6,7,8,9,10],
                         [11,12,13, 14,15,16,17,18,19,20],[1,2,3,4,5,6,7,8,9,10],
                         [11,12,13, 14,15,16,17,18,19,20]])
    randDataY = np.array([1,0,1,1,1,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0])
    foldsX = (np.array_split(randDataX, k))
    foldsY = (np.array_split(randDataY, k))
    
    for i in range(k):
        trainX = foldsX.copy() # you wanna work on a copy of your array
        trainY = foldsY.copy()
        testX = foldsX[i]
        testY = foldsY[i]
        del trainX[i]
        del trainY[i]
        
        trainX = np.concatenate(trainX,axis=0)
        trainY = np.concatenate(trainY,axis=0)
        
        print(trainX[0])

[ 1  2  3  4  5  6  7  8  9 10]
[ 1  2  3  4  5  6  7  8  9 10]
